In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from gensim.models import Word2Vec
from sklearn.metrics import classification_report
import numpy as np



In [35]:
# Prendre en main les données
data = pd.read_csv("../data/train.csv")
data

,doc_id,titre,type,difficulte,cout,ingredients,recette
0,recette_221358.xml,"Feuilleté de saumon et de poireau, sauce aux c...",Plat principal,Facile,Moyen,- 1 gros pavé de saumon - 100 g de crevettes d...,Couper finement le blanc et un peu de vert des...
1,recette_48656.xml,Cake poulet/moutarde/amandes,Entrée,Très facile,Bon marché,- 3 œufs - 150 g de farine - 1 sachet de levur...,"Couper finement l'échalote, la faire revenir à..."
2,recette_30049.xml,Bûche à la truite fumée (7ème rencontre),Entrée,Moyennement difficile,Assez Cher,- 800 g de filet de truite saumonnée fumée en ...,Faites blanchir les épinards à l'eau bouillant...
3,recette_71424.xml,Gâteau au yaourt au coco sans huile de laetitia,Dessert,Très facile,Bon marché,- 1 pot de yaourt - 1 pot de lait de coco - 3 ...,Mélanger dans l'ordre tous les ingrédients en ...
4,recette_217204.xml,Crêpes au canard laqué,Entrée,Moyennement difficile,Moyen,- 90 g de farine - 45 g de maïzena - 2 œufs - ...,"Fouetter les œufs avec l'eau, le lait et le su..."
...,...,...,...,...,...,...,...
12468,recette_88545.xml,Tarte thon et courgettes,Plat principal,Facile,Bon marché,- 1 pâte feuilleté - 2 œufs - 2 courgettes moy...,"Couper les courgettes en fines rondelles, les ..."
12469,recette_36440.xml,Gratin de légumes à la viande,Plat principal,Facile,Bon marché,- 500 g de pommes de terre - 400 g de carotte...,Epluchez les pommes de terre et lavez-les. Cou...
12470,recette_174633.xml,Tagliatelles fraiches au Montepulciano,Plat principal,Difficile,Moyen,- Pour les tagliatelles : - 150 g de semoule f...,Je conseille de préparer les tagliatelles la v...
12471,recette_43030.xml,"Bavarois praliné pistache, coeur de mousse au ...",Dessert,Difficile,Assez Cher,- Pour le biscuit Dacquoise : - 160 g de blanc...,Petits commentaires :J'ai indiqué les recettes...


In [36]:
data.columns


Index(['doc_id', 'titre', 'type', 'difficulte', 'cout', 'ingredients',
       'recette'],
      dtype='object')

In [37]:
# Methode Baseline

y = data['type']
data['combined'] = data['titre'] + data['recette']
X = data['combined']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

dummy = DummyClassifier(strategy='stratified')
dummy.fit(X_train, y_train)

y_pred = dummy.predict(X_test)

print(classification_report(y_test, y_pred))


                precision    recall  f1-score   support

       Dessert       0.28      0.29      0.29      1092
        Entrée       0.25      0.22      0.24       912
Plat principal       0.46      0.47      0.46      1738

      accuracy                           0.36      3742
     macro avg       0.33      0.33      0.33      3742
  weighted avg       0.36      0.36      0.36      3742



In [80]:
### Run2: TF-IDF

In [70]:
data = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")


data['combined'] = data['titre'] + data['recette']

X = data['combined']
y = data['type']

test['combined'] = test['titre'] + test['recette']
X_testt = test['combined']
y_testt = test['type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

tfidf = TfidfVectorizer()

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_testt)

svm = SVC(kernel='linear', C=1, random_state=42)

svm.fit(X_train_tfidf, y_train)

y_pred = svm.predict(X_test_tfidf)
print(classification_report(y_testt, y_pred)) 

                precision    recall  f1-score   support

       Dessert       0.98      0.99      0.98       407
        Entrée       0.76      0.69      0.72       337
Plat principal       0.85      0.88      0.86       644

      accuracy                           0.87      1388
     macro avg       0.86      0.85      0.86      1388
  weighted avg       0.86      0.87      0.86      1388



In [50]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Compute evaluation metrics
accuracy = accuracy_score(y_testt, y_pred)
precision = precision_score(y_testt, y_pred, average='weighted')
recall = recall_score(y_testt, y_pred, average='weighted')
f1 = f1_score(y_testt, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_testt, y_pred)
class_report = classification_report(y_testt, y_pred)

# Print the evaluation metrics
print("Evaluation Metrics:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Evaluation Metrics:
Accuracy: 0.8652737752161384
Precision: 0.8627504141389174
Recall: 0.8652737752161384
F1 Score: 0.8634434024360103
Confusion Matrix:
[[403   1   3]
 [  5 232 100]
 [  5  73 566]]
Classification Report:
                precision    recall  f1-score   support

       Dessert       0.98      0.99      0.98       407
        Entrée       0.76      0.69      0.72       337
Plat principal       0.85      0.88      0.86       644

      accuracy                           0.87      1388
     macro avg       0.86      0.85      0.86      1388
  weighted avg       0.86      0.87      0.86      1388



In [40]:

stopwords = nltk.corpus.stopwords.words('french')

def clean_text(text):
    text = re.sub(r'<.*?>', '', text) #Remove HTML tags
    text = re.sub(r'[^a-zA-Z]', ' ', text) #Remove punctuation
    text = text.lower() #Convert to lowercase
    text = text.split() #Split the text
    text = [word for word in text if not word in set(stopwords)] #Remove stopwords
    text = ' '.join(text)
    return text

def stemmer(text):
    text = text.split()
    stemmer = nltk.PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in text]
    text = ' '.join(stemmed_words)
    return text

In [81]:
### Run3: Word2Vec


In [71]:
 #Modèle Word2Vec

test = pd.read_csv("../data/test.csv")
test['combined'] = test['titre'] + test['recette']
X_testt = test['combined']
y_testt = test['type']



from sklearn.svm import SVC
corpus = [sentence.split() for sentence in X_train]
word2vec_model = Word2Vec(corpus, vector_size=100, window=5, min_count=1, sg=1)

# Calcul des vecteurs de mots moyens pour chaque document
def document_vector(word2vec_model, doc):
    """Calculer le vecteur moyen pour un document en utilisant les vecteurs de mots Word2Vec"""
    # filtrer les mots absents dans le vocabulaire
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    if len(doc) != 0:
        return np.mean(word2vec_model.wv[doc], axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

X_train_word2vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_train])
X_test_word2vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_testt])

svm_classifier = SVC(kernel='linear', C=1, random_state=42)
svm_classifier.fit(X_train_word2vec, y_train)




y_pred_word2vec = svm_classifier.predict(X_test_word2vec)

print("Classification Report Word2Vec:")
print(classification_report(y_testt, y_pred_word2vec))


Classification Report Word2Vec:
                precision    recall  f1-score   support

       Dessert       0.95      0.99      0.97       407
        Entrée       0.77      0.46      0.58       337
Plat principal       0.78      0.92      0.84       644

      accuracy                           0.83      1388
     macro avg       0.83      0.79      0.80      1388
  weighted avg       0.82      0.83      0.81      1388



In [82]:
### Run4: CountVectorizer

In [83]:
## Modèle CountVectorizer
test = pd.read_csv("../data/test.csv")
test['combined'] = test['titre'] + test['recette']
X_testt = test['combined']
y_testt = test['type']

vectorizer = CountVectorizer()

X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_testt)

svm_classifier = SVC(random_state=42)
svm_classifier.fit(X_train_counts, y_train)

y_pred_counts = svm_classifier.predict(X_test_counts)

# Print the classification report
print("Classification Report CountVectorizer:")
print(classification_report(y_testt, y_pred_counts))

Classification Report CountVectorizer:
                precision    recall  f1-score   support

       Dessert       0.96      0.99      0.98       407
        Entrée       0.77      0.63      0.69       337
Plat principal       0.83      0.89      0.86       644

      accuracy                           0.86      1388
     macro avg       0.85      0.84      0.84      1388
  weighted avg       0.85      0.86      0.85      1388



In [67]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Compute evaluation metrics
accuracy = accuracy_score(y_testt, y_pred_counts)
precision = precision_score(y_testt, y_pred_counts, average='weighted')
recall = recall_score(y_testt, y_pred_counts, average='weighted')
f1 = f1_score(y_testt, y_pred_counts, average='weighted')
conf_matrix = confusion_matrix(y_testt, y_pred_counts)
class_report = classification_report(y_testt, y_pred_counts)

# Print the evaluation metrics
print("Evaluation Metrics:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Evaluation Metrics:
Accuracy: 0.8523054755043228
Precision: 0.8478986016196004
Recall: 0.8523054755043228
F1 Score: 0.8474590818283543
Confusion Matrix:
[[403   2   2]
 [ 11 206 120]
 [  7  63 574]]
Classification Report:
                precision    recall  f1-score   support

       Dessert       0.96      0.99      0.97       407
        Entrée       0.76      0.61      0.68       337
Plat principal       0.82      0.89      0.86       644

      accuracy                           0.85      1388
     macro avg       0.85      0.83      0.84      1388
  weighted avg       0.85      0.85      0.85      1388



In [ ]:
######### TEST TF-IDF avec Nettotage de texte et Stemming   #########

In [66]:
# Prétraitement des données d'entraînement
data['combined'] = data['titre'] + ' ' + data['recette']
data['combined'] = data['combined'].apply(clean_text)
data['combined'] = data['combined'].apply(stemmer)

# Prétraitement des données de test
test['combined'] = test['titre'] + ' ' + test['recette']
test['combined'] = test['combined'].apply(clean_text)
test['combined'] = test['combined'].apply(stemmer)
X_testt = test['combined']
y_testt = test['type']

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data['combined'], data['type'], test_size=0.3, random_state=42)

# Vecteur TF-IDF
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_testt)  # Utiliser transform, pas fit_transform

# Entraînement du modèle SVM
svm = SVC(kernel='linear', C=1, random_state=42)
svm.fit(X_train_tfidf, y_train)

# Prédiction sur les données de test
y_pred = svm.predict(X_test_tfidf)

# Évaluation du modèle
print(classification_report(y_testt, y_pred))


[nltk_data] Downloading package stopwords to /home/lahad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                precision    recall  f1-score   support

       Dessert       0.97      1.00      0.98       407
        Entrée       0.79      0.69      0.74       337
Plat principal       0.85      0.89      0.87       644

      accuracy                           0.87      1388
     macro avg       0.87      0.86      0.86      1388
  weighted avg       0.87      0.87      0.87      1388



In [ ]:
####################### TEST Hyperpramètres #######################

In [72]:
## Word2Vec
from sklearn.model_selection import GridSearchCV

# Définir les paramètres à tester
parameters = {'kernel':('linear', 'rbf'), 'C':[0.1, 1, 10]}

# Créer l'objet GridSearchCV
clf = GridSearchCV(svm_classifier, parameters)

# Entraîner le modèle
clf.fit(X_train_word2vec, y_train)

# Prédiction sur les données de test
y_pred_word2vec = clf.predict(X_test_word2vec)

# Évaluation du modèle
print("Classification Report Word2Vec:")
print(classification_report(y_testt, y_pred_word2vec))

# Afficher les meilleurs paramètres
print("Best Parameters:")
print(clf.best_params_)

Classification Report Word2Vec:
                precision    recall  f1-score   support

       Dessert       0.95      0.99      0.97       407
        Entrée       0.76      0.50      0.60       337
Plat principal       0.79      0.91      0.85       644

      accuracy                           0.83      1388
     macro avg       0.83      0.80      0.81      1388
  weighted avg       0.83      0.83      0.82      1388

Best Parameters:
{'C': 10, 'kernel': 'rbf'}
